Code generation mostly came around from the tedious nature of making your own custom types and registering them to a cluster's scheme. In this way code generation came about with some tools that we can use; the popular one being a shell script called *generate-groups.sh*. An invocation to that script would look something like so

In [ ]:
$ vendor/k8s.io/code-generator/generate-groups.sh all \
 github.com/programming-kubernetes/cnat/cnat-client-go/pkg/generated
 github.com/programming-kubernetes/cnat/cnat-client-go/pkg/apis \
 cnat:v1alpha1 \
 --output-base "${GOPATH}/src" \
 --go-header-file "hack/boilerplate.go.txt"

For a description about the parameters look at page 126. Other than the parameters to the shell script, you can modify the generation of your code by using tags. Tags are go comments that you place within your go files and look like this: 
- // +some-tag
- // +someOtherTag=value
These tags fall into two different categories:
- Local tags which are usually placed above type declartions i.e. something like your structs
- Global tags which you'll find / place in the *doc.go* file above the package line
    - The tag *// +groupName=example.com* defines the fully qualified API group name and **needs to be included if Go parent package name does not match the groupName**. This is important because the typed client will not be able to generate the correct HTTP path unless the group package is correct or the tag is defined. 
Some tags are also position sensitive i.e. for some tags you have to be very particular about where you define them; they might have to be one line directly above your type declaration or exactly one line above that; this comes from historical reasons so don't worry about it too much and **look it up if you have questions**. This tag – *// +k8s:deepcopy-gen:interfaces=k8s.io/apimachinery/pkg/runtime.Object* – is important because it allows for the go code to be generic. Since we're using interfaces and specifically the runtime.Object interface, the deep copy method is essential. The tag – *// +genclient* – is also very important as this is the tag that will actually generate the client for your CR. You can add configs to this tag by appending a colon along with your desire i.e. *// +genclient:noStatus* or *// +genclient:nonNamespaced* (creates a client for cluster scoped resources). 